# LSTM Models for Stock Price Prediction

The provided notebook outlines the process of creating and training Long Short-Term Memory (LSTM) models for stock price prediction. Here's a brief description of the notebook:

Imports: The notebook begins by importing necessary libraries such as Pandas, NumPy, Matplotlib, scikit-learn, TensorFlow, and yfinance.

Data Fetching: A function fetch_stock_data is defined to fetch historical stock data from Yahoo Finance using the yfinance library.

Data Preprocessing: Another function preprocess_data preprocesses and normalizes the fetched data using the MinMaxScaler from scikit-learn.

Sequence Creation: The create_sequences function creates sequences for LSTM training by splitting the data into input sequences and corresponding output labels.

Model Building: The build_train_lstm_model function builds and trains an LSTM model using TensorFlow's Keras API.

Future Predictions: The make_future_predictions function makes future predictions using the trained model.

Model Evaluation: The evaluate_model function evaluates the model's performance on the test set and visualizes the results using Matplotlib.

Main Process: The main process is orchestrated by the process_stock function, which fetches data, preprocesses it, trains the model, evaluates its performance, makes future predictions, and creates a DataFrame to store the results.

Top Stocks List: A list of top stocks in NYSE is defined.

Model Training: The notebook iterates over each stock in the list, trains an LSTM model, and stores the results in a dictionary.

Output Saving: The final DataFrame with predictions is saved to a CSV file in the 'LSTM Model Predictions' folder. Additionally, each trained model is saved in the 'LSTM Models' folder.

Results Display: Finally, the notebook displays or saves the results, including mean absolute error on the test set and the DataFrame with predictions for each stock.

This notebook provides a comprehensive workflow for training LSTM models for stock price prediction and demonstrates how to save the models and predictions for future use.

In [4]:
# Create your virtual environment 
# Update Pip using python.exe -m pip install --upgrade pip from terminal
# Execute the pip install command
!pip install pandas numpy matplotlib scikit-learn tensorflow yfinance
!pip install --upgrade tensorflow


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\aksha\\OneDrive\\Documents\\LHL\\Stock Prediction App\\Notebooks\\Model Training Code\\model_training_venv\\Lib\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\tsi\\alts\\zero_copy_frame_protector\\alts_grpc_integrity_only_record_protocol.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths




   ---------------------------------------- 0.0/300.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/300.9 MB 1.4 MB/s eta 0:03:41
   ---------------------------------------- 0.2/300.9 MB 2.6 MB/s eta 0:01:54
   ---------------------------------------- 0.7/300.9 MB 6.4 MB/s eta 0:00:47
   ---------------------------------------- 1.6/300.9 MB 10.0 MB/s eta 0:00:30
   ---------------------------------------- 2.1/300.9 MB 11.3 MB/s eta 0:00:27
   ---------------------------------------- 2.6/300.9 MB 11.2 MB/s eta 0:00:27
    --------------------------------------- 4.1/300.9 MB 14.6 MB/s eta 0:00:21
    --------------------------------------- 5.3/300.9 MB 16.3 MB/s eta 0:00:19
    --------------------------------------- 6.4/300.9 MB 16.5 MB/s eta 0:00:18
    --------------------------------------- 7.4/300.9 MB 17.0 MB/s eta 0:00:18
   - -------------------------------------- 7.8/300.9 MB 16.0 MB/s eta 0:00:19
   - -------------------------------------- 9.2/300.9 MB 17.2 

### Import the packages

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import yfinance as yf
from datetime import datetime
import os

C:\Users\aksha\AppData\Local\Temp\ipykernel_23220\177761964.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Function to fetch historical stock data from Yahoo Finance
def fetch_stock_data(stock_symbol, start_date, end_date):
    data = yf.download(stock_symbol, start=start_date, end=end_date)
    return data['Close'].values.reshape(-1, 1), data.index

# Function to preprocess and normalize the data
def preprocess_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Function to create sequences for LSTM training
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, 0])
        y.append(data[i+seq_length, 0])
    return np.array(X), np.array(y)

# Function to build and train the LSTM model
def build_train_lstm_model(X_train, y_train, epochs=50):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=epochs, batch_size=32)
    return model

# Function to make future predictions with the trained model
def make_future_predictions(model, last_data, scaler, seq_length, future_days):
    future_predictions = []

    for _ in range(future_days):
        input_sequence = last_data[-seq_length:]
        input_sequence = np.reshape(input_sequence, (1, seq_length, 1))
        forecasted_price = model.predict(input_sequence)
        last_data = np.append(last_data, forecasted_price)
        future_predictions.append(scaler.inverse_transform(forecasted_price)[0, 0])

    return future_predictions

# Function to evaluate and visualize the model performance on the test set
def evaluate_model(model, X_test, y_test, scaler, date_index, plots_folder):
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform([y_test])

    mae = mean_absolute_error(y_test.flatten(), predictions.flatten())
    
    # Visualize the results with actual dates on the x-axis
    plt.figure(figsize=(12, 6))
    
    # Convert date_index to ndarray
    date_index = date_index.to_numpy()

    # Plotting
    plt.plot(date_index[-len(y_test.flatten()):], y_test.flatten(), label='Actual Prices', color='blue')
    plt.plot(date_index[-len(predictions.flatten()):], predictions.flatten(), label='Predicted Prices', color='red')
    plt.title('Stock Price Prediction with LSTM')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()

    # Save the plot
    if not os.path.exists(plots_folder):
        os.makedirs(plots_folder)

    plt.savefig(os.path.join(plots_folder, f'{stock_symbol}_prediction_plot.png'))

    plt.close()  # Close the plot to avoid displaying it
    
    return mae, predictions

# Main function to orchestrate the entire process for each stock
def process_stock(stock_symbol, start_date, end_date, seq_length, future_days, parent_directory):
    # Fetch historical stock data
    data, date_index = fetch_stock_data(stock_symbol, start_date, end_date)

    # Preprocess and normalize the data
    scaled_data, scaler = preprocess_data(data)

    # Create sequences for LSTM training
    X, y = create_sequences(scaled_data, seq_length)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # Build and train the LSTM model
    model = build_train_lstm_model(X_train, y_train)

    # Specify the folder path for saving plots
    plots_folder = os.path.join(parent_directory, 'LSTM Model Performance Plots')

    # Evaluate the model and save the plot
    mae, predictions = evaluate_model(model, X_test, y_test, scaler, date_index, plots_folder)

    # Retrain the model on the entire dataset
    model = build_train_lstm_model(X, y)

    # Make future predictions
    future_predictions = make_future_predictions(model, scaled_data, scaler, seq_length, future_days)

    # Create a DataFrame for the output
    dates_future = pd.date_range(start=date_index[-1], periods=future_days + 1)[1:]  # Start from the day after the last existing date
    dates = np.concatenate([date_index, dates_future])
    actual_prices = np.concatenate([data.flatten(), np.full(future_days, np.nan)])
    forecasted_prices = np.full(len(dates), np.nan)
    forecasted_prices[-len(predictions):] = predictions.flatten()
    forecasted_prices[-future_days:] = future_predictions

    # Create the DataFrame
    result_df = pd.DataFrame({
        'Date': dates,
        'Actual Prices': actual_prices,
        'Forecasted Prices': forecasted_prices
    })

    return result_df, mae, model

# List of top 10 stocks in NYSE
top_n_stocks = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'BRK-A', 'JNJ', 'GOOGL']

# Specify date range for historical data
start_date = "2012-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')  # Set end_date to the current system date

# Specify parameters for LSTM model
seq_length = 10
future_days = 120 
# These predictions were generated as on 13th Feb, so forecasts shall be 120 days post that.  
# Same are saved in the LSTM Model Predictions folder. 
# In the streamlit app when the user has to predict the forecasted price, the models will be 
# loaded again, and using the sequence length given in the code, fresh predictions will be made. 
# Please do not confuse the predictions in this folder with that of streamlit app. The ones
# happening in streamlit app will be real time predictions generated from the recent most
# seq length of that time series. 

# Get the absolute path of the current working directory
current_directory = os.path.abspath('')

# Navigate to the parent directory
parent_directory = os.path.dirname(os.path.dirname(current_directory))


# Process each stock and store results in a dictionary
results_dict = {}

for stock_symbol in top_n_stocks:
    result_df, mae, model = process_stock(stock_symbol, start_date, end_date, seq_length, future_days, parent_directory)
    results_dict[stock_symbol] = {'DataFrame': result_df, 'MAE': mae, 'Model': model}

# Save the final DataFrame to a CSV file
predictions_folder = os.path.join(parent_directory, 'LSTM Model Predictions')

# Create the folder if it doesn't exist
if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)

# Save the prediction set in the "predictions" folder
for stock_symbol, results in results_dict.items():
    results['DataFrame']['MAE'] = results['MAE']
    results['DataFrame'].to_csv(os.path.join(predictions_folder, f'{stock_symbol}_forecast.csv'), index=False)
    

# Create a directory to save models if it doesn't exist
models_dir = os.path.join(parent_directory, 'LSTM Models')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Save each model
for stock_symbol, results in results_dict.items():
    model_path = os.path.join(models_dir, f'{stock_symbol}_model.tf')
    results['Model'].save(model_path)

# Display or save the results as needed
for stock_symbol, results in results_dict.items():
    print(f"\nResults for {stock_symbol}:\n")
    print(f"Mean Absolute Error on Test Set: {results['MAE']:.4f}\n")
    print(results['DataFrame'])

c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\Notebooks\Model Training Code\model_training_venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed




Epoch 1/50

76/76 [==============================] - 3s 9ms/step - loss: 0.0082
Epoch 2/50
76/76 [==============================] - 1s 9ms/step - loss: 1.4701e-04
Epoch 3/50
76/76 [==============================] - 1s 9ms/step - loss: 1.1068e-04
Epoch 4/50
76/76 [==============================] - 1s 9ms/step - loss: 1.0372e-04
Epoch 5/50
76/76 [==============================] - 1s 9ms/step - loss: 1.0059e-04
Epoch 6/50
76/76 [==============================] - 1s 9ms/step - loss: 1.0150e-04
Epoch 7/50
76/76 [==============================] - 1s 10ms/step - loss: 9.8309e-05
Epoch 8/50
76/76 [==============================] - 1s 9ms/step - loss: 1.0100e-04
Epoch 9/50
76/76 [==============================] - 1s 9ms/step - loss: 1.0123e-04
Epoch 10/50
76/76 [==============================] - 1s 9ms/step - loss: 9.8934e-05
Epoch 11/50
76/76 [==============================] - 1s 9ms/step - loss: 9.0007e-05
Epoch 12/50
76/76 [==============================] - 1s 10ms/step - loss: 9.2589e-05


c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\Notebooks\Model Training Code\model_training_venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


Epoch 1/50
76/76 [==============================] - 5s 15ms/step - loss: 0.0087
Epoch 2/50
76/76 [==============================] - 1s 14ms/step - loss: 7.7851e-05
Epoch 3/50
76/76 [==============================] - 1s 15ms/step - loss: 4.9108e-05
Epoch 4/50
76/76 [==============================] - 1s 14ms/step - loss: 4.8435e-05
Epoch 5/50
76/76 [==============================] - 1s 13ms/step - loss: 4.9633e-05
Epoch 6/50
76/76 [==============================] - 1s 13ms/step - loss: 5.0194e-05
Epoch 7/50
76/76 [==============================] - 1s 14ms/step - loss: 4.9159e-05
Epoch 8/50
76/76 [==============================] - 1s 14ms/step - loss: 4.9389e-05
Epoch 9/50
76/76 [==============================] - 1s 15ms/step - loss: 4.9631e-05
Epoch 10/50
76/76 [==============================] - 1s 15ms/step - loss: 4.8196e-05
Epoch 11/50
76/76 [==============================] - 1s 14ms/step - loss: 4.8957e-05
Epoch 12/50
76/76 [==============================] - 1s 11ms/step - loss: 4.78

c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\Notebooks\Model Training Code\model_training_venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


...

INFO:tensorflow:Assets written to: c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\LSTM Models\JNJ_model.tf\assets


INFO:tensorflow:Assets written to: c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\LSTM Models\JNJ_model.tf\assets


INFO:tensorflow:Assets written to: c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\LSTM Models\GOOGL_model.tf\assets


INFO:tensorflow:Assets written to: c:\Users\aksha\OneDrive\Documents\LHL\Stock Prediction App\LSTM Models\GOOGL_model.tf\assets



Results for AAPL:

Mean Absolute Error on Test Set: 3.3106

           Date  Actual Prices  Forecasted Prices       MAE
0    2012-01-03      14.686786                NaN  3.310625
1    2012-01-04      14.765714                NaN  3.310625
2    2012-01-05      14.929643                NaN  3.310625
3    2012-01-06      15.085714                NaN  3.310625
4    2012-01-09      15.061786                NaN  3.310625
...         ...            ...                ...       ...
3162 2024-06-07            NaN         234.490082  3.310625
3163 2024-06-08            NaN         233.975571  3.310625
3164 2024-06-09            NaN         233.469864  3.310625
3165 2024-06-10            NaN         232.980087  3.310625
3166 2024-06-11            NaN         232.513397  3.310625

[3167 rows x 4 columns]

Results for MSFT:

Mean Absolute Error on Test Set: 5.9089

           Date  Actual Prices  Forecasted Prices       MAE
0    2012-01-03      26.770000                NaN  5.908929
1    2012-01-